# CSE 151A PA2 
Author: Xiaochen Li 

Source of Help: C4.5: Programs for Machine Learning by J. Ross Quinlan

youtube video on D-tree utilizing gini coeff: https://www.youtube.com/watch?v=LDRbO9a6XPU

## Answer
1. Build Decision Tree without pruning. Draw first Three layer
    
    Format ; IS (Feature number) splitting rule ? [number of data]
    
    Predict {label1: number of label 1, label2: number of label 2}

```
Is (F5)  PAYMENT_DELAY_SEPTEMBER < 0.5 ? [2000]
--> True:
  Is (F1)  LIMIT_BAL < 415000.0 ? [1319]
  --> True:
    Is (F17)  PAY_AMT1 < 2506.5 ? [1284]
    --> True:
      Is (F1)  LIMIT_BAL < 75000.0 ? [704]
      --> True:
      --> False:
    --> False:
      Is (F1)  LIMIT_BAL < 25000.0 ? [580]
      --> True:
      --> False:
  --> False:
    Is (F21)  PAY_AMT5 < 208.0 ? [35]
    --> True:
      Predict {1.0: 4.0}
    --> False:
      Is (F17)  PAY_AMT1 < 2174.0 ? [31]
      --> True:
        Predict {0.0: 9.0}
      --> False:
--> False:
  Is (F5)  PAYMENT_DELAY_SEPTEMBER < 1.5 ? [681]
  --> True:
    Is (F20)  PAY_AMT4 < 584.5 ? [292]
    --> True:
      Is (F12)  BILL_AMT2 < 231.5 ? [134]
      --> True:
      --> False:
    --> False:
      Is (F12)  BILL_AMT2 < 1461.0 ? [158]
      --> True:
      --> False:
  --> False:
    Is (F21)  PAY_AMT5 < 2006.0 ? [389]
    --> True:
      Is (F19)  PAY_AMT3 < 2476.0 ? [232]
      --> True:
      --> False:
        Predict {1.0: 50.0}
    --> False:
      Is (F19)  PAY_AMT3 < 13075.0 ? [157]
      --> True:
        Predict {1.0: 147.0}
      --> False:
      
```
2. Training error and Test Error


TRAIN ERROR : 0.0

TEST ERROR: 17.30000000000001

3. Prune the tree

First Prune

VAL ERROR : 12.200000000000003

TEST ERROR: 11.700000000000003

Second Prune

VAL ERROR : 10.700000000000003


TEST ERROR: 10.299999999999997


4. Most Important feature:

PAYMENT_DELAY_SEPTEMBER

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.stats import mode
import copy

In [2]:
## Global Variables

# file path
DATA_PATH = "./data/"
VALIDATE_PATH = "pa2validation.txt"
TRAIN_PATH = "pa2train.txt"
TEST_PATH = "pa2test.txt"
FEATURE_PATH = "pa2features.txt"

# data
TRAIN_DATA = np.genfromtxt(DATA_PATH + TRAIN_PATH)
VALIDATE_DATA = np.genfromtxt(DATA_PATH + VALIDATE_PATH)
TEST_DATA = np.genfromtxt(DATA_PATH + TEST_PATH)
FEATURES = np.loadtxt(DATA_PATH + FEATURE_PATH,dtype=str)


In [3]:
FEATURES
COLS = np.insert(FEATURES,22, "LABEL")
pd.DataFrame(data=TRAIN_DATA, columns=COLS)

,LIMIT_BAL,SEX,COLLEGE_ABOVE,AGE,PAYMENT_DELAY_SEPTEMBER,PAYMENT_DELAY_AUGUST,PAYMENT_DELAY_JULY,PAYMENT_DELAY_JUNE,PAYMENT_DELAY_MAY,PAYMENT_DELAY_APRIL,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,LABEL
0,30000.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30324.0,31124.0,25502.0,2506.0,2430.0,1300.0,1000.0,34436.0,950.0,1.0
1,160000.0,1.0,1.0,2.0,4.0,3.0,2.0,2.0,3.0,2.0,...,103928.0,101540.0,99587.0,0.0,5500.0,6700.0,0.0,27.0,2800.0,1.0
2,190000.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,38827.0,39488.0,40208.0,1900.0,1800.0,1700.0,1600.0,1500.0,1600.0,1.0
3,140000.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2880.0,1914.0,968.0,2000.0,1500.0,3000.0,1914.0,0.0,3287.0,1.0
4,200000.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2040.0,16338.0,2076.0,8290.0,9427.0,2240.0,16000.0,2076.0,833.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,60000.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22816.0,26587.0,22917.0,2000.0,1246.0,720.0,4780.0,1700.0,4000.0,0.0
1996,30000.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,58287.0,29522.0,30103.0,1505.0,1457.0,1016.0,1207.0,1600.0,1000.0,0.0
1997,200000.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5659.0,0.0,0.0,0.0,0.0,5659.0,0.0,1.0
1998,50000.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,390.0,390.0,390.0,390.0,780.0,0.0,390.0,390.0,1170.0,0.0


In [4]:
class Question:
    

    def __init__(self, column, value):
        
        self.column = column
        self.value = value
        
    def match(self, row): 
        return row[self.column] < self.value
        
    def partition(self, df):
        true_df = None
        false_df = None

        true_index = df[:,self.column] < self.value
        false_index = df[:,self.column] >= self.value
        
        true_df = df[true_index]
        false_df = df[false_index]
        return true_df, false_df

    def __repr__(self):

        condition = "<"
        return "Is {} {} {} {} ?".format("("+"F"+str(self.column+1)+") " ,FEATURES[self.column], condition, str(self.value))

In [5]:
def unique_vals(df, col):
    return np.unique(df[:,col])

def class_counts(df):
    return np.unique(df[:,-1], return_counts=True)


def myEntropy(df):
    labels, counts = class_counts(df)
    probs = counts / float(df.shape[0])
    ent = - np.sum(probs * np.log(probs))
    return ent

def info_gain(left, right, current_entropy):
    p = float(left.shape[0]) / (left.shape[0] + right.shape[0]) ## Pr{Z = 0}
    return current_entropy - p * myEntropy(left) - (1 - p) * myEntropy(right) ## H(x)- H(x|z)


def find_best_split(df): 
    
    ## remeber the best IG 
    best_IG = 0  
    best_question = None 
    
    current_entropy = myEntropy(df)
    
    # Loop over each feature and its value to find best info_gain
    n_features = df.shape[1] - 1  
    for col in range(n_features): 
        values = np.sort(np.unique(df[:,col])) 
        
        ## calculate the mean of every consecutive values
        thresholds = [(a + b) / 2 for a, b in zip(values[::1], values[1::1])] 
        
        for threshold in thresholds:  

            question = Question(col, threshold)
            true_rows, false_rows = question.partition(df)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            IG = info_gain(true_rows, false_rows, current_entropy)
            
            if IG > best_IG:
                
                best_IG, best_question = IG, question

    return best_IG, best_question

def accuracy(preds, labels):
    return np.mean(preds == labels) * 100


def print_tree(node,depth, spacing=""):

    # Base case: we've reached a leaf
    #if isinstance(node, Leaf):
    if (node.isleaf):
        print (spacing + "Predict", node.predictions)
        return
    if depth == 0 :
        return
    
    # Print the question at this node
    print (spacing + str(node.question) + " [" + str(node.size) + "]" )

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch,depth-1, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch,depth -1, spacing + "  ")

In [6]:
class Node:

    def __init__(self, question, true_branch, false_branch,size,rows, isleaf , parent):
        self.df = rows
        self.size = size
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.parent = parent
        
        
        
        counts = np.column_stack(class_counts(rows))
        self.predictions = {row[0]:row[1] for row in counts}
        self.isleaf = isleaf
   

In [7]:
class Leaf:


    def __init__(self, rows):
        counts = np.column_stack(class_counts(rows))
        self.predictions = {row[0]:row[1] for row in counts}

In [23]:
class ID3:
    
    def __init__(self):
        pass
        
    def train(self, df):
        self.root = self._build_tree(df,None)
        
        
    def _build_tree(self, df, parent):
        
        # Pick best question by maximizing information gain
        IG, question = find_best_split(df)
        # If there is not gain, or we have reached max depth
        if IG == 0:
            return Node(None, None, None,0, df, True, parent)
        
        true_branch = Node(None,None,None,0,df,False,parent)
        false_branch = Node(None,None,None,0,df,False,parent)
        # Partition and recursively build left and right branches
        true_rows, false_rows = question.partition(df)
        true_branch = self._build_tree(true_rows,true_branch)
        false_branch = self._build_tree(false_rows,false_branch)

        return Node(question, true_branch, false_branch, df.shape[0], df, False, parent)
    
    def predict(self, row):
        
        prediction = self._classify(row, self.root)
        return prediction
    
    def _classify(self, row, node):
        
        # Base case: At a leaf node
        #if isinstance(node, Leaf):
        if (node.isleaf):
            maxcount = 0
            maxlabel = None
            for k, v in node.predictions.items(): ## dictionary like
                if v >= maxcount:
                    maxcount = v
                    maxlabel = k
            return maxlabel

        # If not, match with question.
        if node.question.match(row):
            return self._classify(row, node.true_branch)
        else:
            return self._classify(row, node.false_branch) 
        
    def test(self, df, labels):
        # Not vectorized.
        
        preds = np.zeros(df.shape[0])
        for i, row in enumerate(df):
            preds[i] = self.predict(row)
            
        return accuracy(preds.astype(int), labels.astype(int)) 
    
    def prune(self, validation,allowed_second):
        
        true_df = np.array([
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
        
        ])
    
        false_df = np.array([
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        ])
        
        
        i = 0;
        queue = [self.root]
        while len(queue) > 0:
            if ( i == 2 ): break
            
            old_err = self.test(validation[:,:-1], validation[:,-1])
            print(old_err)
            
            
            curNode = queue.pop(0);
            temp = copy.deepcopy(curNode.true_branch)
            if (mode(curNode.true_branch.df[:,-1])[0][0] == 0):
                curNode.true_branch = Node(None, None, None,0, false_df, True, None)
            else:
                curNode.true_branch = Node(None, None, None,0, true_df, True, None)
            
            
            new_err = self.test(validation[:,:-1], validation[:,-1])
            i+=1
            print(i,"prune")
            print(new_err)
            if (new_err < old_err):
                curNode.true_branch = temp
                print(i,"prune failed")
                i-=1
                
            if not (allowed_second):
                return 
            
            
            old_err = new_err
            temp = copy.deepcopy(curNode.false_branch)
            
            if (mode(curNode.false_branch.df[:,-1])[0][0] == 0):
                curNode.false_branch = Node(None, None, None,0, false_df, True, None)
            else:
                curNode.false_branch = Node(None, None, None,0, true_df, True, None)
            print(old_err)
            i+=1
            print(i,"prune")
            
            new_err = self.test(validation[:,:-1], validation[:,-1])
            print(new_err)
            if (new_err < old_err):
                curNode.false_branch = temp
                print(i,"prune failed")
                i-=1
                
            
            if not (curNode.true_branch.isleaf):
                queue.append(curNode.true_branch)
            if not (curNode.false_branch.isleaf):
                queue.append(curNode.false_branch)
            
            


In [24]:
id3 = ID3()
id3.train(TRAIN_DATA)

In [25]:
train_err_no_prune = 100 -id3.test(TRAIN_DATA[:,:-1], TRAIN_DATA[:,-1])
test_err_no_prune = 100 - id3.test(TEST_DATA[:,:-1], TEST_DATA[:,-1])

print("TRAIN ERROR : " + str(train_err_no_prune))
print("TEST ERROR: " + str(test_err_no_prune))

TRAIN ERROR : 0.0
TEST ERROR: 17.30000000000001


In [26]:
print_tree(id3.root,4)

Is (F5)  PAYMENT_DELAY_SEPTEMBER < 0.5 ? [2000]
--> True:
  Is (F1)  LIMIT_BAL < 415000.0 ? [1319]
  --> True:
    Is (F17)  PAY_AMT1 < 2506.5 ? [1284]
    --> True:
      Is (F1)  LIMIT_BAL < 75000.0 ? [704]
      --> True:
      --> False:
    --> False:
      Is (F1)  LIMIT_BAL < 25000.0 ? [580]
      --> True:
      --> False:
  --> False:
    Is (F21)  PAY_AMT5 < 208.0 ? [35]
    --> True:
      Predict {1.0: 4.0}
    --> False:
      Is (F17)  PAY_AMT1 < 2174.0 ? [31]
      --> True:
        Predict {0.0: 9.0}
      --> False:
--> False:
  Is (F5)  PAYMENT_DELAY_SEPTEMBER < 1.5 ? [681]
  --> True:
    Is (F20)  PAY_AMT4 < 584.5 ? [292]
    --> True:
      Is (F12)  BILL_AMT2 < 231.5 ? [134]
      --> True:
      --> False:
    --> False:
      Is (F12)  BILL_AMT2 < 1461.0 ? [158]
      --> True:
      --> False:
  --> False:
    Is (F21)  PAY_AMT5 < 2006.0 ? [389]
    --> True:
      Is (F19)  PAY_AMT3 < 2476.0 ? [232]
      --> True:
      --> False:
        Predict {1.0: 50.0}


In [29]:
id3.prune(VALIDATE_DATA, False) ##only one prune allowed

train_err_prune_1 = 100 -id3.test(VALIDATE_DATA[:,:-1], VALIDATE_DATA[:,-1])
test_err_prune_1 = 100 - id3.test(TEST_DATA[:,:-1], TEST_DATA[:,-1])

print("VAL ERROR : " + str(train_err_prune_1))
print("TEST ERROR: " + str(test_err_prune_1))

87.8
1 prune
87.8
VAL ERROR : 12.200000000000003
TEST ERROR: 11.700000000000003


In [30]:
id3.train(TRAIN_DATA)
id3.prune(VALIDATE_DATA, True) 

82.1
1 prune
87.8
87.8
2 prune
89.3


In [31]:
train_err_prune_1 = 100 -id3.test(VALIDATE_DATA[:,:-1], VALIDATE_DATA[:,-1])
test_err_prune_1 = 100 - id3.test(TEST_DATA[:,:-1], TEST_DATA[:,-1])

print("VAL ERROR : " + str(train_err_prune_1))
print("TEST ERROR: " + str(test_err_prune_1))

VAL ERROR : 10.700000000000003
TEST ERROR: 10.299999999999997


In [13]:
id3.test(TEST_DATA[:,:-1], TEST_DATA[:,-1])

89.7

In [32]:
print_tree(id3.root,100)

Is (F5)  PAYMENT_DELAY_SEPTEMBER < 0.5 ? [2000]
--> True:
  Predict {0: 1}
--> False:
  Predict {1: 1}
